In [5]:
import vaex
# Load feather file
dataset = vaex.open("small_file.feather")
#df = vaex.from_dataset(dataset)
dataset

#,Unnamed: 0,A,B,C,D
0,RKKJ4iV0mc,-1.678956391558604,-0.2388790445093592,1.1055515208624067,0.9397256114058312
1,64R71Tc6GH,-1.0613047340422994,0.0012908630027789,0.7597418636728487,-1.3354472500844283
2,y0XbFIBeYQ,-0.52860517089301,-1.5631492451216775,0.7313482419249768,1.2812339151048822
3,So3KGoCNAQ,-0.8838850384514394,0.8719733291672772,-0.0509251167295204,0.0539034128672671
4,izkdOcbuaS,0.0749033580995846,0.6177090150056431,-0.3673956758004302,1.2586992706145634
...,...,...,...,...,...
25,oIrhXhmANk,0.3383776747285827,-0.8376922103228934,1.4450365118973951,0.0264008002984321
26,Kglh1F05gs,-0.6317361020640085,-1.5202307662996857,1.4112140712473706,-0.0628700909456976
27,4XICHPjeXg,-1.5979936778741917,0.2818413305536353,-0.1776103251863211,-1.3838726191119473
28,pfUSiWhz3m,0.2053849849266507,-1.0447514966816651,1.630160479565647,0.5472991792256756


In [6]:
type(dataset)

vaex.dataframe.DataFrameLocal

In [31]:
def csv_to_feather_function(input_file):
    import filepattern
    import argparse, logging
    import numpy as np
    import typing, os
    from pathlib import Path
    import vaex
    
    # Import environment variables
    POLUS_LOG = getattr(logging,os.environ.get('POLUS_LOG','INFO'))

    # Initialize the logger
    logging.basicConfig(format='%(asctime)s - %(name)-8s - %(levelname)-8s - %(message)s',
                        datefmt='%d-%b-%y %H:%M:%S')
    logger = logging.getLogger("main")
    logger.setLevel(POLUS_LOG)

    def csv2parquet(input_path: Path,
                    output_path: Path
                    ) -> np.ndarray:
        """Convert a csv file to Apache feather file
        """
        print("input_path: ", input_path)
        # Get the number of columns in the csv
        with open(input_path,'r') as fr:
            ncols = len(fr.readline().split(','))

        # Set the chunk size to get roughly 16MB at a time, or at least 1 row
        chunk_size = max([2**24 // ncols,1])

        # Load the file and convert to hdf5 to make reading scalable
        logger.info('reading file...')
        csv = vaex.from_csv(input_path,convert=True,chunk_size=chunk_size)

        # Write to parquet file
        logger.info('writing file...')
        csv.export_feather(output_path)
        csv.close()

        # Remove intermediate files
        logger.info('removing intermediates...')
        os.unlink(input_path.with_suffix('.csv.yaml'))
        os.unlink(input_path.with_suffix('.csv.hdf5'))
        logger.info('done.')

    def main(input_file,
             outDir: Path,
             ) -> None:
        """ Main execution function

        All functions in your code must have docstrings attached to them, and the
        docstrings must follow the Google Python Style:
        https://www.sphinx-doc.org/en/master/usage/extensions/example_google.html
        """

        input_file = Path(input_file)
        print("input_file before: ", input_file)
        print("type: ", type(input_file))
        output_file = Path(outDir).joinpath(input_file.name.replace('.csv','.feather'))

        logger.info(f'Converting file: {input_file.name}')
        print("after: input_file: ", input_file)
        csv2parquet(input_file, output_file)

    if __name__=="__main__":

        ''' Argument parsing '''
        logger.info("Parsing arguments...")
        parser = argparse.ArgumentParser(prog='main', description='Convert data from csv to feather.')


        #: Input csv collection to be processed by this plugin
        logger.info('input_file = {}'.format(input_file))
        #: Output collection
        outDir = ""

        main(input_file, outDir)

In [32]:
from pathlib import Path
#: This function takes in a csv a adds a feather file
csv_to_feather_function(Path("small_file1.csv"))

05-Aug-21 04:42:00 - main - INFO - Parsing arguments...
05-Aug-21 04:42:00 - main - INFO - input_file = small_file1.csv
05-Aug-21 04:42:00 - main - INFO - Converting file: small_file1.csv
05-Aug-21 04:42:00 - main - INFO - reading file...
05-Aug-21 04:42:00 - main - INFO - writing file...
05-Aug-21 04:42:00 - main - INFO - removing intermediates...
05-Aug-21 04:42:00 - main - INFO - done.


input_file before:  small_file1.csv
type:  <class 'pathlib.PosixPath'>
after: input_file:  small_file1.csv
input_path:  small_file1.csv


In [ ]:
# Load UCI census train and test data into dataframes.
import pandas as pd
features = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status",
            "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
            "Hours per week", "Country", "Target"]
train_data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=features,
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")
test_data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    names=features,
    sep=r'\s*,\s*',
    skiprows=[0],
    engine='python',
    na_values="?")

In [ ]:
# Calculate the feature statistics proto from the datasets and stringify it for use in facets overview.

# This code assumes that the facets-overview package has been installed through pip,
# along with a tensorflow (or tensorflow-gpu) package.
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'train', 'table': train_data},
                                  {'name': 'test', 'table': test_data}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

In [ ]:
# Display the facets overview visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))